In [98]:
import pandas as pd
import re
from google.colab import drive
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


# Word processing libraries
#from nltk.corpus import wordnet
#import string
#from nltk import pos_tag
#from nltk.corpus import stopwords
#from nltk.tokenize import WhitespaceTokenizer

import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize 
from string import punctuation # Importing list of punctuations from string


import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import remove_stopwords,STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
np.random.seed(2018)


from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV


from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC, SVC
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier, GradientBoostingClassifier

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [99]:
# you 1st need to right click on the shared folder and select 'Add to my Drive'
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [100]:
data=pd.read_csv("/content/drive/MyDrive/omdena/BP_NLP/Project1/game_reviews.csv")
data

,author age,recommended age,review
0,"Teen, 17 years old",age 7+,Kids dont listent to the parents saying ''ThIs...
1,"Kid, 11 years old",age 2+,I have been playing this game for many years a...
2,"Kid, 12 years old",age 7+,The game is great with no true inappropriate t...
3,"Teen, 13 years old",age 5+,Are you sure you got common sense. I would giv...
4,"Kid, 10 years old",age 5+,IDK WHAT TO SAY BUT DIS IS DA BEST GAME EVA
...,...,...,...
556,"Teen, 14 years old",age 7+,"Wow. Just wow. Common Sense, THIS GAME IS IN A..."
557,"Teen, 13 years old",age 4+,A quite popular gem on the app store/google pl...
558,"Kid, 12 years old",age 2+,"It's a abhorrent try by Mojang, though a excel..."
559,"Teen, 15 years old",age 6+,Minecraft: Pocket Edition is an app which you ...


In [101]:
#create an age column
data["age"]= data["author age"].str.extract('(\d+)').astype("float")

In [102]:
data['author age']=data['author age'].str.lower()
data['recommended age']=data['recommended age'].str.lower()
data['review']=data['review'].str.lower()
data


,author age,recommended age,review,age
0,"teen, 17 years old",age 7+,kids dont listent to the parents saying ''this...,17.0
1,"kid, 11 years old",age 2+,i have been playing this game for many years a...,11.0
2,"kid, 12 years old",age 7+,the game is great with no true inappropriate t...,12.0
3,"teen, 13 years old",age 5+,are you sure you got common sense. i would giv...,13.0
4,"kid, 10 years old",age 5+,idk what to say but dis is da best game eva,10.0
...,...,...,...,...
556,"teen, 14 years old",age 7+,"wow. just wow. common sense, this game is in a...",14.0
557,"teen, 13 years old",age 4+,a quite popular gem on the app store/google pl...,13.0
558,"kid, 12 years old",age 2+,"it's a abhorrent try by mojang, though a excel...",12.0
559,"teen, 15 years old",age 6+,minecraft: pocket edition is an app which you ...,15.0


In [103]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 561 entries, 0 to 560
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   author age       561 non-null    object 
 1   recommended age  561 non-null    object 
 2   review           526 non-null    object 
 3   age              561 non-null    float64
dtypes: float64(1), object(3)
memory usage: 17.7+ KB


In [104]:
data.rename(columns={'author age': 'authors_age', 'recommended age': 'recommended_age'}, inplace=True)

In [105]:
#function to create new labeled column
def label(age):
  if age < 10:
    return "not suitable"
  else:
    return "suitable"

#function to get first word
def word(x):
  return x.split(",")[0]

# Define the function to implement POS tagging:
def get_wordnet_pos(pos_tag):
  if pos_tag.startswith('J'):
      return wordnet.ADJ
  elif pos_tag.startswith('V'):
      return wordnet.VERB
  elif pos_tag.startswith('N'):
      return wordnet.NOUN
  elif pos_tag.startswith('R'):
      return wordnet.ADV
  else:
      return wordnet.NOUN


# Define the main function to clean text in various ways:
def clean_text(text):
  # 2. remove URLs
  text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', '', text)
    
  # 5. Convert text to lowercase
  text = text.lower()
    
  # 6. tokenize text and remove punctuation
  text = [word.strip(string.punctuation) for word in text.split(" ")]
    
  # 7. remove numbers
  text = [word for word in text if not any(c.isdigit() for c in word)]
    
  # 8. remove stop words
  stop = stopwords.words('english')
  text = [x for x in text if x not in stop]
    
  # 9. remove empty tokens
  text = [t for t in text if len(t) > 0]
    
  # 10. pos tag text and lemmatize text
  pos_tags = pos_tag(text)
  text = [WordNetLemmatizer().lemmatize(t[0], get_wordnet_pos(t[1])) for t in pos_tags]
    
  # 11. remove words with only one letter
  text = [t for t in text if len(t) > 1]
    
  # join all
  text = " ".join(text)
    
  return (text)


def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS:
            result.append(lemmatize_stemming(token))
    return str(result)




def crossvalidate_classifier(model, X, y, cm=False):
    
  scores = cross_val_score(model, X, y, scoring='f1_macro', cv=5)
  #scores = cross_val_score(model, X, y, scoring='accuracy', cv=5)
  print("%0.2f f-1 score with a standard deviation of %0.2f" % (scores.mean(), scores.std()))
  y_pred = cross_val_predict(model, X, y, cv=5)

  if(cm):
    conf_mat = confusion_matrix(y, y_pred, labels=['fulfillment','other'])
    plot_confusion_matrix(conf_mat, classes = ['fulfillment','other'])
    
def evaluate_classifier(model, X_train, X_test, y_train, y_test):
      
  # ... fit your model here ...
  model.fit(X_train,y_train)

  # Run predict on your tfidf test data to get your predictions
  pred = model.predict(X_test)

    # Calculate your accuracy using the metrics module
  acc_score = metrics.accuracy_score(pred, y_test)
  print("Accuracy Score:   %0.3f" % acc_score)
    
  f1score = metrics.f1_score(pred, y_test, average='macro')
  print("F-1 Score:   %0.3f" % f1score)

  # Calculate the confusion matrices for the tfidf_svc model
  #svc_cm = metrics.confusion_matrix(y_test, pred, labels=['fulfillment','other'])

  # Plot the confusion matrix using the plot_confusion_matrix function
  #plot_confusion_matrix(svc_cm, classes = ['fulfillment','other'], title="Confusion Matrix")
    
  return model

def best_hyperparam(X_train_data, X_test_data, y_train_data, y_test_data, 
                       model, param_grid, cv=5, scoring_fit='f1_score',
                       do_probabilities = False):
  gs = RandomizedSearchCV(estimator=model,
        param_grid=param_grid, 
        cv=cv, 
        n_jobs=-1, 
        scoring=scoring_fit,
        verbose=2
    )
  fitted_model = gs.fit(X_train_data, y_train_data, epochs=100)
    
  if do_probabilities:
    pred = fitted_model.predict_proba(X_test_data)
  else:
    pred = fitted_model.predict(X_test_data)
    
    return fitted_model, pred

In [106]:
#Creating the label 
#data["label"]=data.apply(lambda x:label(x["age"]),axis=1)

In [107]:
data["review_owner"]=data['authors_age'].apply(lambda x:word(x))

In [108]:
data

,authors_age,recommended_age,review,age,label,review_owner
0,"teen, 17 years old",age 7+,kids dont listent to the parents saying ''this...,17.0,suitable,teen
1,"kid, 11 years old",age 2+,i have been playing this game for many years a...,11.0,suitable,kid
2,"kid, 12 years old",age 7+,the game is great with no true inappropriate t...,12.0,suitable,kid
3,"teen, 13 years old",age 5+,are you sure you got common sense. i would giv...,13.0,suitable,teen
4,"kid, 10 years old",age 5+,idk what to say but dis is da best game eva,10.0,suitable,kid
...,...,...,...,...,...,...
556,"teen, 14 years old",age 7+,"wow. just wow. common sense, this game is in a...",14.0,suitable,teen
557,"teen, 13 years old",age 4+,a quite popular gem on the app store/google pl...,13.0,suitable,teen
558,"kid, 12 years old",age 2+,"it's a abhorrent try by mojang, though a excel...",12.0,suitable,kid
559,"teen, 15 years old",age 6+,minecraft: pocket edition is an app which you ...,15.0,suitable,teen


In [109]:
#subsetting important features
data1= data.copy()
data1['feature'] = data1['authors_age'] +" "+data1['review']
data1= data1[['authors_age','feature','review_owner']]
data1.columns= ['author_category','feature_description', 'class']

In [110]:
data1['class'].value_counts()

kid     319
teen    242
Name: class, dtype: int64

In [111]:
#rechecking data
data1

,author_category,feature_description,class
0,"teen, 17 years old","teen, 17 years old kids dont listent to the pa...",teen
1,"kid, 11 years old","kid, 11 years old i have been playing this gam...",kid
2,"kid, 12 years old","kid, 12 years old the game is great with no tr...",kid
3,"teen, 13 years old","teen, 13 years old are you sure you got common...",teen
4,"kid, 10 years old","kid, 10 years old idk what to say but dis is d...",kid
...,...,...,...
556,"teen, 14 years old","teen, 14 years old wow. just wow. common sense...",teen
557,"teen, 13 years old","teen, 13 years old a quite popular gem on the ...",teen
558,"kid, 12 years old","kid, 12 years old it's a abhorrent try by moja...",kid
559,"teen, 15 years old","teen, 15 years old minecraft: pocket edition i...",teen


In [112]:
#removing Nan rows from dataset
df_nlp = data1[~data1.feature_description.isna()]
df_nlp

,author_category,feature_description,class
0,"teen, 17 years old","teen, 17 years old kids dont listent to the pa...",teen
1,"kid, 11 years old","kid, 11 years old i have been playing this gam...",kid
2,"kid, 12 years old","kid, 12 years old the game is great with no tr...",kid
3,"teen, 13 years old","teen, 13 years old are you sure you got common...",teen
4,"kid, 10 years old","kid, 10 years old idk what to say but dis is d...",kid
...,...,...,...
556,"teen, 14 years old","teen, 14 years old wow. just wow. common sense...",teen
557,"teen, 13 years old","teen, 13 years old a quite popular gem on the ...",teen
558,"kid, 12 years old","kid, 12 years old it's a abhorrent try by moja...",kid
559,"teen, 15 years old","teen, 15 years old minecraft: pocket edition i...",teen


In [113]:
#creating the stemming object
stemmer = SnowballStemmer('english')

In [114]:
#cleaning dataset and creating tokens
df_nlp['tokens'] = df_nlp['feature_description'].map(preprocess)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [115]:
df_nlp['class'].value_counts()

kid     303
teen    223
Name: class, dtype: int64

In [116]:
#creating the target 
X = df_nlp['tokens']
y= df_nlp['class']

In [117]:
# Split training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=53, test_size=0.25, shuffle=True, stratify=y )

In [118]:
y_test.head()

472     kid
200     kid
66     teen
457     kid
128    teen
Name: class, dtype: object

In [119]:
# Initialize BoW with count vectorizer
count_vectorizer = CountVectorizer(stop_words='english', max_df=0.9, min_df=0.05)

# Create count train and test variables
count_train = count_vectorizer.fit_transform(X_train)
count_test = count_vectorizer.transform(X_test)

# Initialize term frequency with tfidf vectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_df=0.9, min_df=0.05)

# Create tfidf train and test variables
tfidf_train = tfidf_vectorizer.fit_transform(X_train)
tfidf_test = tfidf_vectorizer.transform(X_test)

In [120]:
print(count_train.shape, tfidf_train.shape)

(394, 86) (394, 86)


In [121]:
print(count_vectorizer.vocabulary_)

{'kid': 32, 'minecraft': 42, 'best': 4, 'game': 25, 'use': 76, 'play': 55, 'think': 72, 'creativ': 15, 'mode': 44, 'age': 0, 'surviv': 68, 'violent': 79, 'zombi': 85, 'kill': 33, 'anim': 2, 'food': 22, 'parent': 51, 'hit': 29, 'sword': 69, 'start': 66, 'friend': 23, 'teen': 70, 'fun': 24, 'need': 48, 'good': 26, 'build': 9, 'want': 80, 'lot': 40, 'like': 38, 'version': 77, 'better': 5, 'updat': 75, 'let': 36, 'server': 65, 'love': 41, 'say': 63, 'real': 58, 'world': 82, 'peopl': 54, 'pocket': 57, 'edit': 19, 'easi': 18, 'great': 27, 'thing': 71, 'educ': 20, 'hous': 30, 'learn': 35, 'differ': 17, 'blood': 8, 'littl': 39, 'children': 13, 'bite': 6, 'die': 16, 'recommend': 59, 'violenc': 78, 'way': 81, 'block': 7, 'mob': 43, 'time': 73, 'stuff': 67, 'encourag': 21, 'monster': 46, 'red': 60, 'scari': 64, 'pc': 52, 'bad': 3, 'player': 56, 'worri': 83, 'child': 12, 'resourc': 61, 'know': 34, 'buy': 10, 'life': 37, 'turn': 74, 'onlin': 49, 'amaz': 1, 'peac': 53, 'money': 45, 'hard': 28, 'mult

In [122]:
#Evaluating the model on BoW test set
crossvalidate_classifier(RandomForestClassifier(), count_train, y_train)

1.00 f-1 score with a standard deviation of 0.01


In [123]:
#Evaluating the model on TfiDf test set
crossvalidate_classifier(RandomForestClassifier(), tfidf_train, y_train)

1.00 f-1 score with a standard deviation of 0.01


In [124]:
crossvalidate_classifier(GradientBoostingClassifier(), tfidf_train, y_train)

0.99 f-1 score with a standard deviation of 0.01
